# 0. Import Data

In [7]:
%reload_ext tensorboard
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('Churn.csv')

In [9]:
# Elimina Churn y customer id de las variables independientes
X = pd.get_dummies(df.drop(['Churn', 'Customer ID'], axis=1))
# Transformar la variable Churn Yes a 1 y No a 0 
y = df['Churn'].apply(lambda x: 1 if x=='Yes' else 0)

In [10]:
#Separa X e Y en dos muestras una para entrenar el modelo y otro para testear modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [11]:
y_train.head()

2779    0
6053    0
1514    1
1914    0
460     0
Name: Churn, dtype: int64

# 1. Import Dependencies

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

# 2. Build and Compile Model

In [13]:
#Crear modelo secuencial
model = Sequential()
#Agregar capas
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

2022-08-22 12:54:07.026650: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

# 3. Fit, Predict and Evaluate

In [17]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir log_dir

model.fit(X_train, y_train, epochs=100, batch_size=32,validation_data=(X_test,y_test),callbacks=[tensorboard_callback])


Epoch 1/100
177/177 [==============================] - 2s 10ms/step - loss: 0.4201 - accuracy: 0.8002 - val_loss: 0.4207 - val_accuracy: 0.7977
Epoch 2/100
177/177 [==============================] - 2s 12ms/step - loss: 0.4197 - accuracy: 0.8018 - val_loss: 0.4618 - val_accuracy: 0.7757
Epoch 3/100
177/177 [==============================] - 2s 14ms/step - loss: 0.4217 - accuracy: 0.7973 - val_loss: 0.4240 - val_accuracy: 0.7949
Epoch 4/100
177/177 [==============================] - 2s 12ms/step - loss: 0.4216 - accuracy: 0.8012 - val_loss: 0.4242 - val_accuracy: 0.7956
Epoch 5/100
177/177 [==============================] - 2s 11ms/step - loss: 0.4207 - accuracy: 0.8025 - val_loss: 0.5798 - val_accuracy: 0.7239
Epoch 6/100
177/177 [==============================] - 2s 11ms/step - loss: 0.4203 - accuracy: 0.7989 - val_loss: 0.4735 - val_accuracy: 0.7644
Epoch 7/100
177/177 [==============================] - 2s 12ms/step - loss: 0.4215 - accuracy: 0.8028 - val_loss: 0.4422 - val_accuracy:

In [ ]:
y_hat = model.predict(X_test,callbacks=[tensorboard_callback])
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

45/45 [==============================] - 1s 13ms/step


In [ ]:
accuracy_score(y_test, y_hat)

0.7757274662881476

# 4. Saving and Reloading

In [ ]:
model.save('tfmodel')

INFO:tensorflow:Assets written to: tfmodel/assets


In [ ]:
del model 

In [ ]:
model = load_model('tfmodel')